In [7]:
# import dependencies 
import requests 
import pandas as pd
import numpy as np
import time 
from datetime import datetime
from sqlalchemy import create_engine
import psycopg2
import sqlite3
import config

In [ ]:
#get base fucntion 

In [73]:
#define fuction to streamline conversion to dataframe for testing 
def reframe(type_input, query):
    #sq fro sqlite
    if  type_input == "sq":
        df = pd.DataFrame(sqlite_cur.execute(query).fetchall())
        df.columns = [i[0] for i in sqlite_cur.description]
    #ps for postgre
    elif type_input == "ps":
        df = pd.DataFrame(cur.execute(query).fetchall())
        df.columns = [i[0] for i in cur.description]
    return df

In [3]:
conn = psycopg2.connect(database="postgres", user=config.db_user, password=config.db_password, sslmode="disable")
cur = conn.cursor();

In [4]:
cur.execute('''SELECT * FROM information_schema.tables where table_schema = 'public' ''')

In [5]:
pd.DataFrame((cur.fetchall()))

,0,1,2,3,4,5,6,7,8,9,10,11
0,postgres,public,notes,BASE TABLE,None,None,None,None,None,YES,NO,None


In [6]:
conn.close()

In [35]:
sqlite_conn = sqlite3.connect("../sqlite/database.sqlite")

In [9]:
sqlite_cur = sql.cursor()

In [75]:
reframe("sq","SELECT * FROM sqlite_master WHERE type='table'")

,type,name,tbl_name,rootpage,sql
0,table,reviews,reviews,2,"CREATE TABLE reviews (\n\treviewid INTEGER,\n\..."
1,table,artists,artists,3,"CREATE TABLE artists (\n\treviewid INTEGER, ar..."
2,table,genres,genres,4,"CREATE TABLE genres (\n\treviewid INTEGER, gen..."
3,table,labels,labels,5,"CREATE TABLE labels (\n\treviewid INTEGER, lab..."
4,table,years,years,6,"CREATE TABLE years (\n\treviewid INTEGER, year..."
5,table,content,content,7,"CREATE TABLE content (\n\treviewid INTEGER, co..."


In [ ]:
#print table info 
#loop througj info to understand schema 